In [1]:
from dgp import DGP_Finite
from inference import Inference2, Inference
import numpy as np
import pandas as pd 

# generate multuple sample size, multiple model designs, multiple inference method,
# a single estimator
dgp = DGP_Finite('1',1000)
Yobs, D, A = dgp.get_data()
inf = Inference2(Yobs, D, A, dgp.cluster, dgp.tuple_idx, dgp.tau10)
print(inf.inference('mp'), inf.inference('robust'), inf.inference('clustered'))

def cover_rate(sample_size=1000, modelY='1', ntrials=2000):
    dgp = DGP_Finite(modelY, sample_size)
    cover = np.zeros((ntrials, 3))
    for i in range(2000):
        Yobs, D, A = dgp.get_data()
        inf = Inference2(Yobs, D, A, dgp.cluster, dgp.tuple_idx, dgp.tau10)
        cover[i,0] = inf.inference('mp')
        cover[i,1] = inf.inference('robust')
        cover[i,2] = inf.inference('clustered')
    #tmp = cover[:,0] - cover[:,2]
    #print(tmp[tmp>0], cover[:,0][tmp>0], cover[:,2][tmp>0])
    #print(np.mean(tmp>0))
    return np.mean(cover, axis=0)

0 1 0


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/apply.py:290: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  results[i] = self.f(v)


In [2]:
modelYs = ['1','2','3','4','5','6']
sample_sizes = [50, 100, 200, 500, 1000]
modelYs = ['1']
sample_sizes = [1000]

In [3]:
output = np.zeros((len(modelYs)*3,len(sample_sizes)))
for i, modelY in enumerate(modelYs):
    for j, n in enumerate(sample_sizes):
        cover = cover_rate(n, modelY)
        output[i*3:i*3+3,j] = cover
print(output)
pd.DataFrame(output).to_csv("finite_pop_cover_rate.csv")

[[0.962 ]
 [1.    ]
 [0.9585]]


In [ ]:
from joblib import Parallel, delayed
import multiprocessing

sample_sizes = [40, 80, 160, 480, 1000]

qk_pairs = [(q,k) for q in modelYs for k in sample_sizes]
def processInput(qk):
    q, k = qk
    cover = cover_rate(k, q)
    return (q,k,cover)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in qk_pairs)
output = np.zeros((len(modelYs)*3,len(sample_sizes)))
for (q,k,cover) in results:
    i = int(q)-1
    j = sample_sizes.index(k)
    output[i*3:i*3+3,j] = cover
print(output)
pd.DataFrame(output).to_csv("finite_pop_cover_rate.csv")